# Recherche d'information : librairie PyTerrier

Dans cette partie, nous nous intéressons à la librairie [PyTerrier](https://pyterrier.readthedocs.io/en/latest/#) qui permet de mettre en place diverses briques d'un moteur de recherche.
PyTerrier est basée sur [Terrier](http://terrier.org/) qui est un moteur de recherche développé en Java.

Nous allons voir : 
*   l'installation et la configuration
*   l'indexation d'une collection
*   l'accès à l'index
*   l'évaluation d'un moteur de recherche


## Installation ete configuration

Après l'installation de la librairie, il est nécessaire d'initialiser PyTerrier pour importer les fichiers jar et démarrer la machine virtuelle associée; 

In [1]:
!pip install python-terrier

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

     |████████████████████████████████| 97 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 24.1 MB/s 
     |████████████████████████████████| 69 kB 9.2 MB/s 
     |████████████████████████████████| 255 kB 53.5 MB/s 
     |████████████████████████████████| 41 kB 369 kB/s 
     |████████████████████████████████| 6.4 MB 22.9 MB/s 
     |████████████████████████████████| 126 kB 48.5 MB/s 
     |████████████████████████████████| 1.2 MB 55.9 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 596 kB 9.9 MB/s 
     |████████████████████████████████| 294 kB 54.5 MB/s 
     |████████████████████████████████| 291 kB 57.6 MB/s 
     |████████████████████████████████| 59 kB 5.7 MB/s 
     |████████████████████████████████| 45 kB 1.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=d8af9b957063c30ce676a2b1e5658e107f22c2bc4f600b8c3aabf72b84ff3726
  Stored in directory: /root/.cach

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



## Indexation d'une collection

Il est possible d'indexer plusieurs formats de collection : format TREC, fichiers en texte brut ou en PDF, ou encore des Dataframe Pandas ([pour plus de détails](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html)).

Un petit exemple à titre illustratif est foourni dans le code suivant :

In [2]:
import pandas as pd

# configuration de l'affichage
pd.set_option('display.max_colwidth', 150)

# le jeu de données au format DataFrame
docs_df = pd.DataFrame([
        ["d1", "this is the first document of many documents"],
        ["d2", "this is another document"],
        ["d3", "the topic of this document is unknown"]
    ], columns=["docno", "text"])

# création de l'index
indexer = pt.DFIndexer("./index_3docs", overwrite=True)         # Définition du format de données (DFIndexer())
index_ref = indexer.index(docs_df["text"], docs_df["docno"])    # Indexation des champs text et docno
!ls -lh index_3docs/                                            # Affichage de l'index sauvegardé dans "./index_3docs/"

total 40K
-rw-r--r-- 1 root root    3 Mar 10 13:05 data.direct.bf
-rw-r--r-- 1 root root   51 Mar 10 13:05 data.document.fsarrayfile
-rw-r--r-- 1 root root    4 Mar 10 13:05 data.inverted.bf
-rw-r--r-- 1 root root  344 Mar 10 13:05 data.lexicon.fsomapfile
-rw-r--r-- 1 root root  249 Mar 10 13:05 data.lexicon.fsomaphash
-rw-r--r-- 1 root root   33 Mar 10 13:05 data.meta-0.fsomapfile
-rw-r--r-- 1 root root   24 Mar 10 13:05 data.meta.idx
-rw-r--r-- 1 root root   48 Mar 10 13:05 data.meta.zdata
-rw-r--r-- 1 root root 4.1K Mar 10 13:05 data.properties


De nombreux fichiers sont créés : index direct, index inverse, méta-données de l'index et de la configuration de l'indexation, etc...


Il est également possible de modifier la configuration de l'indexation : [voir ici](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#indexing-configuration).

Pour chager un index existant en local :

In [3]:
index = pt.IndexFactory.of(index_ref)

Il est aussi possible de voir les statistiques de l'index. 
pour connaître toutes les fonctions d'interrogation, se référencer à la [javadoc](http://terrier.org/docs/current/javadoc/org/terrier/structures/Index.html).

In [4]:
# statistiques de la collection
print(index.getCollectionStatistics().toString())

Number of documents: 3
Number of terms: 4
Number of postings: 6
Number of fields: 0
Number of tokens: 7
Field names: []
Positions:   false



In [5]:
# statistiques du vocabulaire.
# Nt : document frequency : nombre de documents contenant le terme
# TF : term frequency : nombre d'occurences du terme
# maxTF : nombre d'occurences maximal pour un document
for kv in index.getLexicon():
  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

document (<class 'str'>) -> term0 Nt=3 TF=4 maxTF=2 @{0 0 0} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
first (<class 'str'>) -> term1 Nt=1 TF=1 maxTF=1 @{0 0 7} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
topic (<class 'str'>) -> term2 Nt=1 TF=1 maxTF=1 @{0 1 1} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
unknown (<class 'str'>) -> term3 Nt=1 TF=1 maxTF=1 @{0 1 5} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)


In [6]:
# focus sur un terme particulier
index.getLexicon()["document"].toString()

'term0 Nt=3 TF=4 maxTF=2 @{0 0 0}'

In [7]:
# récupère les statistiques de l'index inverse à partir d'un terme particulier
pointer = index.getLexicon()["document"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

ID(0) TF(2) doclen=3
ID(1) TF(1) doclen=1
ID(2) TF(1) doclen=3



De plus, PyTerrier met à disposition [une collection de jeux de données pré-traités](https://pyterrier.readthedocs.io/en/latest/datasets.html).
Dans ce qui suit, nous allons nous concentrer sur le jeu de données CORD19 qui recense des articles liés à la crise sanitaire Covid-19. Il est

In [8]:
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
  # création de l'index. Utilisation de l'itérateur pour parcourir la collection
  indexer = pt.index.IterDictIndexer(pt_index_path)

  # on donne à l'index la fonction pour parcourir l'index avec l'itérateur  get_corpus_iter() 
  # On spécifie les champs à indexer et les meta-données à sauvegarder
  index_ref = indexer.index(cord19.get_corpus_iter(), 
                            fields=('abstract',), 
                            meta=('docno',))

else:
  # dans le cas où l'index existe déjà
  index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 [00:00<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M [00:00<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 24.6k/269M [00:00<21:45, 206kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 57.3k/269M [00:00<19:42, 228kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 90.1k/269M [00:00<19:08, 234kB/s]
https://ai2-semanticscholar-cord-1

cord19/trec-covid documents:   0%|          | 0/192509 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  


13:10:00.023 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
13:11:25.115 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 54937 empty documents
13:11:25.212 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing

**Exercice 1**

Afficher les statistiques de l'index Cord19 et analyser statistiques du terme "tv" (pas trop fréquent pour question d'affichage).

In [16]:
index.getLexicon()["tv"].toString()
# Nt --> nombre de document dans lequel le terme apparait
# TF --> nombre d'apparition du terme dans le corpus
# maxTF --> nombre max d'apparition d'un terme dans un document

'term18759 Nt=70 TF=180 maxTF=2147483647 @{0 18614194 0} TFf=180'

In [20]:
pointer = index.getLexicon()["tv"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

(2777,3,F[3]) doclen=219
(10054,1,F[1]) doclen=258
(11115,3,F[3]) doclen=228
(11396,1,F[1]) doclen=173
(11998,1,F[1]) doclen=70
(15443,2,F[2]) doclen=65
(22159,1,F[1]) doclen=74
(24922,1,F[1]) doclen=51
(29671,2,F[2]) doclen=165
(32156,1,F[1]) doclen=259
(38532,7,F[7]) doclen=166
(42346,5,F[5]) doclen=236
(43113,10,F[10]) doclen=193
(43829,1,F[1]) doclen=203
(49420,2,F[2]) doclen=151
(50340,4,F[4]) doclen=193
(51298,4,F[4]) doclen=146
(51965,1,F[1]) doclen=64
(55321,2,F[2]) doclen=248
(56159,1,F[1]) doclen=227
(56638,4,F[4]) doclen=149
(60575,1,F[1]) doclen=135
(61226,4,F[4]) doclen=245
(61831,1,F[1]) doclen=188
(62509,1,F[1]) doclen=229
(64784,6,F[6]) doclen=159
(69551,5,F[5]) doclen=100
(70746,1,F[1]) doclen=226
(71655,1,F[1]) doclen=163
(71784,1,F[1]) doclen=182
(72685,1,F[1]) doclen=317
(77313,1,F[1]) doclen=193
(77512,1,F[1]) doclen=123
(81426,1,F[1]) doclen=150
(85336,9,F[9]) doclen=165
(86932,3,F[3]) doclen=161
(87660,11,F[11]) doclen=245
(92317,4,F[4]) doclen=358
(92318,4,F[4])

## Recherche de documents à partir de l'index

Pour effectuer une recherche dans l'index, il faut utiliser la fonction batchRetrieve qui prend en paramètre l'index et le modèle de pondération (tf, tf-idf, etc...). La liste des modèles supportés est disponible [ici](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html).

In [21]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,12409,ij3ncdb6,0,21.0,chemical reactions
1,1,24432,094bk0t0,1,13.0,chemical reactions
2,1,11268,qroxmo85,2,12.0,chemical reactions
3,1,120041,pb00yr0r,3,12.0,chemical reactions
4,1,147193,ei4rb8fr,4,12.0,chemical reactions
...,...,...,...,...,...,...
995,1,96252,8clnrlec,995,2.0,chemical reactions
996,1,96758,rvw94nsg,996,2.0,chemical reactions
997,1,97138,lrx94c81,997,2.0,chemical reactions
998,1,98181,w3160mgp,998,2.0,chemical reactions


On récupère alors un DataFrame dont les colonnes sont les suivantes : 
*   qid : identifiant de la requête. Par défaut, il s'agit de "1", puisqu'il s'agit de notre première et unique requête.
*   docid : l'identifiant interne de Terrier pour chaque document
*   docno : l'identifiant unique externe (chaîne de caractères) pour chaque document
*   score : score des documents selon le modèle choisi (ici : fréquence totale des tf des termes de la requête dans chaque document)
*   rank : rang du document dénotant l'ordre décroissant par score.
*   query : la requête d'entrée

In [22]:
# autre exemple de modèle : TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tfidf.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,18717,iavwkdpr,0,11.035982,chemical reactions
1,1,171636,v3blnh02,1,10.329726,chemical reactions
2,1,147193,ei4rb8fr,2,10.317138,chemical reactions
3,1,121217,msdycum2,3,9.653734,chemical reactions
4,1,170863,sj8i9ss2,4,9.500211,chemical reactions
...,...,...,...,...,...,...
995,1,2428,38aabxh1,995,3.790183,chemical reactions
996,1,14752,u709r8ss,996,3.790183,chemical reactions
997,1,20074,wxi1xsbo,997,3.790183,chemical reactions
998,1,117156,ts3obwts,998,3.790183,chemical reactions


On peut aussi fournir plusieurs requêtes grâce à un dataFrame. Pour interroger l'index, on applique la fonction transform() au BatchRetriever (br).
pour plus de détails, voir [les propriétés des transformations](https://pyterrier.readthedocs.io/en/latest/transformer.html) ainsi que les [opérations possibles](https://pyterrier.readthedocs.io/en/latest/operators.html).

In [23]:
import pandas as pd
queries = pd.DataFrame([["q1", "document"], ["q2", "first document"]], columns=["qid", "query"])
br.transform(queries)       # ou aussi : br(queries)

,qid,docid,docno,rank,score,query
0,q1,14569,mm7gxjf1,0,12.0,document
1,q1,137615,duxm9u8v,1,12.0,document
2,q1,18789,x3o3a45b,2,10.0,document
3,q1,189505,hxp258y8,3,10.0,document
4,q1,37445,q5wglpoj,4,9.0,document
...,...,...,...,...,...,...
1995,q2,137347,xye8pgzt,995,3.0,first document
1996,q2,137487,k0y0fz7k,996,3.0,first document
1997,q2,138448,ut3ayexx,997,3.0,first document
1998,q2,138807,fulh7trp,998,3.0,first document


**Exercice 2**

Ordonnancer les documents pour 3 requêtes : "covid disease", "hospital" et "home".
La fonction d'ordonnacement devra être de la forme suivante : 


```
0.4 * score_Bm25 + 0.6 * score_Dirichlet
```


In [117]:
queries = pd.DataFrame([["q1", "covid disease"], ["q2", "hospital"], ["q3", "home"]], columns=["qid", "query"])

dlm = pt.BatchRetrieve(index, wmodel="DirichletLM")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

df1 = dlm.transform(queries)
df2 = bm25.transform(queries)

df1.sort_values(by=['docid'], inplace=True)
df2.sort_values(by=['docid'], inplace=True)

df1.drop(columns=["docno", "rank", "query"], inplace=True)
df2.drop(columns=["docno", "rank", "query"], inplace=True)

res = pd.merge(df1, df2, how ='outer', on =["docid", "qid"])
res["score"] = 0.6 * res["score_x"] + 0.4 * res["score_y"]
res.drop(columns=["score_x", "score_y"], inplace=True)
res.sort_values(by=['score'], inplace=True, ascending=False)

print("ID du meilleur doc:", res["docid"][0])
print(res)

ID du meilleur doc: 152
     qid   docid     score
896   q3   73694  7.256685
308   q3   37219  7.175818
807   q3   71618  7.128688
304   q3   36315  6.907965
2442  q3  160321  6.791849
...   ..     ...       ...
4186  q2  192290       NaN
4187  q3  192310       NaN
4188  q3  192360       NaN
4189  q1  192486       NaN
4190  q1  192500       NaN

[4191 rows x 3 columns]


In [118]:
pipeline = dlm >> bm25
print(pipeline.transform(queries))

linear = 0.4*bm25 + 0.6*dlm
print(linear.transform(queries))

     qid   docid     docno  rank     score          query
0     q1   43498  9d8go2sl     0  6.148625  covid disease
1     q1  106017  a0w3n30z     1  6.148625  covid disease
2     q1  106018  smyp9fyo     2  6.148625  covid disease
3     q1  106019  vtfn9y5p     3  6.148625  covid disease
4     q1  158611  30ff3xg2     4  6.124425  covid disease
...   ..     ...       ...   ...       ...            ...
2995  q3   62102  n5qaaqxn   995  4.065851           home
2996  q3   58015  x7or04u6   996  3.808793           home
2997  q3   74632  i8litb66   997  3.770778           home
2998  q3  163315  i1gdsiw6   998  3.676422           home
2999  q3  160913  o0mhzx3g   999  3.212039           home

[3000 rows x 6 columns]
     qid     docid     docno     score          query  rank
0     q1   43498.0  9d8go2sl  3.067285  covid disease     1
1     q1  106017.0  a0w3n30z  3.067285  covid disease     2
2     q1  106018.0  smyp9fyo  3.067285  covid disease     3
3     q1  106019.0  vtfn9y5p  3.067285 

In [90]:
# TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF")
# DPH = pt.BatchRetrieve(index, wmodel="DPH")
# PL2 = pt.BatchRetrieve(index, wmodel="PL2")
# DLM = pt.BatchRetrieve(index, wmodel="DirichletLM")
# BM25 = pt.BatchRetrieve(index, wmodel="BM25")

# queries = pd.DataFrame([["q1", "covid disease"], ["q2", "hospital"], ["q3", "home"]], columns=["qid", "query"])
  
# pt.Experiment([TF_IDF, DPH, PL2, DLM, BM25], queries, ?????, eval_metrics=["map"])

**Excercice 3**

Créez un ordonnanceur qui effectue les opérations suivantes :
* obtient les 10 documents les mieux notés par fréquence de terme (wmodel="Tf")
* obtenir les 10 documents les mieux notés par TF.IDF (wmodel="TF_IDF")
* ré-ordonne uniquement les documents trouvés dans les DEUX paramètres de recherche précédents en utilisant BM25.

Combien de documents sont récupérés par ce pipeline complet pour la requête "chemical"?

Vérification : le document avec le docno "37771" devrait avoir un score de 12.426309 $ pour la requête "chemical".

In [145]:
querie = "chemical"
tf = pt.BatchRetrieve(index, wmodel="Tf",num_results=10)
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF",num_results=10)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pipeline = (tf & tf_idf) >> bm25
result = pipeline.search(querie)
result

,qid,docid,docno,rank,score,query
0,1,37771,jn5qi1jb,0,12.426309,chemical
1,1,134305,0smev8vt,1,12.292890,chemical
2,1,142104,77c9ohxj,2,12.226076,chemical
3,1,56631,sps45fj5,3,11.642770,chemical
4,1,2524,ifebw24e,4,11.439890,chemical


## Reformulation de requêtes 

Il est également possible de mettre en place des pipelines de [reformulation de requêtes](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [146]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipeline = dph >> bo1 >> dph
pipeline.search("chemical reactions")

,qid,docid,docno,rank,score,query_0,query
0,1,147193,ei4rb8fr,0,20.728032,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
1,1,171636,v3blnh02,1,20.684654,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
2,1,18717,iavwkdpr,2,11.160010,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
3,1,20409,1g9kmpdi,3,10.702739,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
4,1,170863,sj8i9ss2,4,10.347593,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
...,...,...,...,...,...,...,...
995,1,114233,2cxb65j1,995,3.942073,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
996,1,183279,8emqwp2o,996,3.942073,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
997,1,88197,f7g1zacc,997,3.941175,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
998,1,178340,zsp5v8hv,998,3.938286,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...


L'autre solution est de l'intégrer directement dans la fonction d'ordonnancement. Mais la requête reformulée n'est pas visible et la solution précédente fait prendre conscience de la pipeline faite par le système de RI (ranking >> reformulation >> ranking quand on utilise des modèles basés sur la relevance feedback. ou reformulation >> ranking sinon). Plus d'exemples [ici](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [147]:
# modèle DPH avant reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "off"})
pipelineQE.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,171636,v3blnh02,0,11.898228,chemical reactions
1,1,147193,ei4rb8fr,1,11.889920,chemical reactions
2,1,18717,iavwkdpr,2,11.688030,chemical reactions
3,1,170863,sj8i9ss2,3,10.143884,chemical reactions
4,1,121217,msdycum2,4,9.860973,chemical reactions
...,...,...,...,...,...,...
995,1,154582,q4neat3x,995,4.070840,chemical reactions
996,1,171895,yjbcvf3o,996,4.070840,chemical reactions
997,1,14781,0it25j3k,997,4.067507,chemical reactions
998,1,70003,a1nc5k51,998,4.067507,chemical reactions


In [148]:
# modèle DPH après reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "on"})
pipelineQE.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,147193,ei4rb8fr,0,20.728032,chemical reactions
1,1,171636,v3blnh02,1,20.684654,chemical reactions
2,1,18717,iavwkdpr,2,11.160010,chemical reactions
3,1,20409,1g9kmpdi,3,10.702739,chemical reactions
4,1,170863,sj8i9ss2,4,10.347593,chemical reactions
...,...,...,...,...,...,...
995,1,114233,2cxb65j1,995,3.942073,chemical reactions
996,1,183279,8emqwp2o,996,3.942073,chemical reactions
997,1,88197,f7g1zacc,997,3.941175,chemical reactions
998,1,178340,zsp5v8hv,998,3.938286,chemical reactions


### Evaluation d'un système de recherche d'information

Pour évaluer un système de RI, il est nécessaire d'avoir un jeu de données constitué de requêtes et de jugements de pertinence. 

In [99]:
# exemple de 5 requêtes pour cord19
cord19.get_topics(variant='title').head(5)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [14.6MB/s]
/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,qid,query
0,1,coronavirus origin
1,2,coronavirus response to weather changes
2,3,coronavirus immunity
3,4,how do people die from the coronavirus
4,5,animal models of covid 19


In [100]:
# exemple de jugements de pertinence pour les 5 premières requêtes
cord19.get_qrels().head(5)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [00:07] [1.14MB] [145kB/s]


,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4


**Exercice 4**

A partir des requêtes et des jugements de pertinence du jeu de données CORD19, Ecrire le code qui permet d'afficher les résultats de la première requête de Cord19. L'affichage fusionnera les colonnes retournées par le BatchRetriever et les colonnes des qrels (merge sur qid et docno pour rajouter label et iteration au tableau). 

In [149]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipeline = dph >> bo1 >> dph
res = pipeline.search(cord19.get_topics(variant='title')["query"][0])
merged = pd.merge(res, cord19.get_qrels(), how ='inner', on =["docno", "qid"])

merged

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,qid,docid,docno,rank,score,query_0,query,label,iteration
0,1,82224,8ccl9aui,0,11.661505,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,2,1
1,1,84953,ax6v6ham,1,11.430387,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,1,3.5
2,1,45549,8l411r1w,2,9.242285,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,0,1
3,1,175892,zy8qjaai,3,8.763194,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,1,1
4,1,166596,d2knbzhl,4,8.614349,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,0,1
...,...,...,...,...,...,...,...,...,...
464,1,146110,4ywt0yqn,965,5.090643,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,0,4
465,1,75817,55loucvc,972,5.084005,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,2,5
466,1,186503,erntrh3p,992,5.052921,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,0,3
467,1,57809,e6h1qvdk,996,5.044950,coronavirus origin,applypipeline:off coronaviru^1.190927068 origin^1.341147614 phylogenet^0.249566089 analys^0.242429428 sar^0.200830689 spike^0.142504108 nucleocaps...,1,5


Il existe cependant une fonction qui permet de calculer l'efficacité de ces ordonnancements au travers des métriques d'évaluation (map, précision, rappel, ndcg, ...)

In [150]:
pt.Experiment(
    [tfidf],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,name,map,ndcg
0,BR(TF_IDF),0.180002,0.370767


**Exercice 5**

Réaliser une expérience comparant l'expansion de requêtes avec le modèle Bo1 et basé sur la KL-divergence; L'expérience est réalisée sur TREC CORD19 avec le modèle de référence BM25. Vous devrez construire des pipelines appropriées (plus de détails sur [l'expansion](https://pyterrier.readthedocs.io/en/latest/rewrite.html) et les [expérimentations](https://pyterrier.readthedocs.io/en/latest/experiments.html)).

Quelles approches entraînent des augmentations significatives de NDCG et MAP ou autres métriques ?

In [161]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
kl = pt.rewrite.KLQueryExpansion(index)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pipeline1 = bm25 >> bo1 >> bm25
pipeline2 = bm25 >> kl >> bm25

pipeline3 = tfidf >> bo1 >> tfidf
pipeline4 = tfidf >> kl >> tfidf

In [162]:
pt.Experiment(
    [pipeline1],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,name,map,ndcg
0,"Compose(Compose(BR(BM25), <pyterrier.rewrite.Bo1QueryExpansion object at 0x7f436e34a910>), BR(BM25))",0.190398,0.388397


In [163]:
pt.Experiment(
    [pipeline2],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,name,map,ndcg
0,"Compose(Compose(BR(BM25), <pyterrier.rewrite.KLQueryExpansion object at 0x7f436e34ac50>), BR(BM25))",0.193292,0.391679


In [164]:
pt.Experiment(
    [pipeline3],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,name,map,ndcg
0,"Compose(Compose(BR(TF_IDF), <pyterrier.rewrite.Bo1QueryExpansion object at 0x7f436e34a910>), BR(TF_IDF))",0.191959,0.387534


In [165]:
pt.Experiment(
    [pipeline4],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,name,map,ndcg
0,"Compose(Compose(BR(TF_IDF), <pyterrier.rewrite.KLQueryExpansion object at 0x7f436e34ac50>), BR(TF_IDF))",0.19524,0.391334
